In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

import torch.optim as optim
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display,clear_output

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from dataset import GermanCreditDataset
from experiments import Benchmarking
from utils.logger_config import setup_logger
from tqdm import tqdm
from models.wrapper import PYTORCH_MODELS

logger = setup_logger()

In [ ]:
from experiments.counterfactual import *
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from models import PyTorchDNN, PyTorchLinearSVM, PyTorchRBFNet, PyTorchLogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.gaussian_process import GaussianProcessClassifier

dataset = GermanCreditDataset()

In [ ]:
input_dim = dataset.get_dataframe().shape[1] - 1
seed = 0
torch.manual_seed(seed)
Avalues_method = 'max'

counterfactual_algorithms = [
    'DiCE',
    # 'DisCount',
    # 'KNN',
]

experiment = Benchmarking(
    dataset=dataset,
    models=[
        # (BaggingClassifier(),'sklearn'),
        # (GaussianProcessClassifier(),'sklearn'),
        # (XGBClassifier(), 'sklearn'),
        (LGBMClassifier(),'sklearn'),
        # (PyTorchLogisticRegression(input_dim=input_dim), 'PYT'),
        # (PyTorchDNN(input_dim=input_dim), 'PYT'),
        # (PyTorchRBFNet(input_dim=input_dim, hidden_dim=input_dim), 'PYT'),
        # (PyTorchLinearSVM(input_dim=input_dim), 'PYT'),
        # (RandomForestClassifier(), 'sklearn'), 
        # (GradientBoostingClassifier(), 'sklearn'), 
        # (AdaBoostClassifier(), 'sklearn'), 
    ],
    shapley_methods=[
            # "Train_Distri",
            # "Train_OTMatch",
            # "CF_UniformMatch",
            # "CF_RandomMatch",
            "CF_OTMatch",
            "CF_ExactMatch",
    ],
    distance_metrics=[
        'optimal_transport',
        'mean_difference',
        'median_difference',
        'max_mean_discrepancy',
    ],
    md_baseline=False,
)

experiment.train_and_evaluate_models(random_state=seed)
experiment.models_performance()

logger.info("\n\n------Compute Counterfactuals------")
sample_num = 50
model_counterfactuals = {}
for model, model_name in zip(experiment.models, experiment.model_names):
    model_counterfactuals[model_name] = {}

    for algorithm in counterfactual_algorithms:
        if algorithm == 'DisCount' and model_name not in PYTORCH_MODELS:
            logger.info(f'Skipping {algorithm} for {model_name} due to incompatability')
            continue
        logger.info(f'Computing {model_name} counterfactuals with {algorithm}')
        function_name = f"compute_{algorithm}_counterfactuals"
        try:
            func = globals()[function_name]
            model_counterfactuals[model_name][algorithm] = func(
                experiment.X_test,
                model = model,
                target_name = experiment.dataset.target_name,
                sample_num = sample_num,
                experiment=experiment,
            )
        except KeyError:
            logger.info(f"Function {function_name} is not working.")


In [ ]:
logger.info("\n\n------Compute Action Policies------")
experiment.compute_intervention_policies(
    model_counterfactuals=model_counterfactuals,
    Avalues_method=Avalues_method,
);

In [ ]:
from copy import deepcopy

trained_models = deepcopy(experiment.models)
model_name = 'LGBMClassifier'
model_index = experiment.model_names.index(model_name)
model = trained_models[model_index]

In [ ]:
from models import ClassifierWrapper
experiment.md_baseline = True
experiment.models = [
    ClassifierWrapper(classifier=model, backend=model.backend),
]
experiment.shapley_methods=[
    "CF_OTMatch",
    "CF_ExactMatch",
]
experiment.distance_metrics=[
    # 'optimal_transport',
    'mean_difference',
]

In [ ]:
logger.info("\n\n------Evaluating Distance Performance Under Interventions------")
experiment.evaluate_distance_performance_under_interventions(
    intervention_num_list=range(0,201,5),
    trials_num=100,
    replace=False
)

In [ ]:
# import pickle 
# with open(f"pickles/{dataset.name}_experiment_optimality.pickle", "rb") as input_file:
#     experiment = pickle.load(input_file)

In [ ]:
from experiments import plotting

plotting.intervention_vs_distance(experiment, save_to_file=True)

In [ ]:
model_name = 'AdaBoostClassifier'
cf_method = 'DiCE'

In [ ]:
X_factual = experiment.model_counterfactuals[model_name][cf_method]['X_factual']
X_counterfactual = experiment.model_counterfactuals[model_name][cf_method]['X']
model_index = experiment.model_names.index(model_name)
model = experiment.models[model_index]

In [ ]:
from experiments.baseline import OptimalMeanDifference
problem = OptimalMeanDifference(X_factual=X_factual, X_counterfactual=X_counterfactual, model=model)

In [ ]:
intervention_num_list = range(0,201,5)

opt_list = []
for intervention_num in range(0,201,5): 
    if (
        len(opt_list) <= 0
        or opt_list[-1] > 1e-4
    ):
        eta = problem.solve_problem(
            C=intervention_num
        )["eta"]
    else:
        eta = 0
    opt_list.append(eta)

In [ ]:
from experiments.latex import TikzPlotGenerator


data = experiment.distance_results[model_name][cf_method]

plot_generator = TikzPlotGenerator(data)
print(plot_generator.generate_plot_code(
    # metric='optimal_transport',
    # metric='max_mean_discrepancy', 
    metric='mean_difference', 
    # metric='median_difference', 
    methods=[
        # "Train_Distri",
        # "Train_OTMatch",
        # "CF_UniformMatch",
        # "CF_RandomMatch",
        "CF_OTMatch",
        "CF_ExactMatch",
        "optimality",
    ],
    corrected_y_lists=[opt_list]
))

In [ ]:
# for e in list(zip(intervention_num_list, opt_list)):
#     print(e)